In [1]:
# loading necessary packages
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import random
random.seed(42)
torch.manual_seed(0)

In [2]:
# Defining Datasets for train and validation
train_mnist_set = torchvision.datasets.MNIST(root='./data',download=True,train=True)
test_mnist_set = torchvision.datasets.MNIST(root='./data',download=True,train=False)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
import PIL

In [4]:
from IPython.display import Image

In [5]:
# Defining a custom dataloader
class CustomDataLoader(Dataset):
  def __init__(self,dataset):
    self.mnist_set = dataset

  def __getitem__(self,index):
    random_int = random.randint(0,9)
    random_int = random_int/9
    image,label = self.mnist_set[index]
    label = label/9
    image = transforms.ToTensor()(image)
    # image = torch.FloatTensor(image)
    image = image.reshape((image.shape[0],-1))
    b = torch.FloatTensor([[random_int]])
    b.reshape((1,-1))
    c = torch.cat((image,b),1)
    gt = [label,random_int+label]
    gt = torch.FloatTensor(gt)
    return c,gt
  
  def __len__(self):
    return len(self.mnist_set)

In [6]:
# Setting tran and val set using dataloader
train_set = CustomDataLoader(train_mnist_set)
val_set = CustomDataLoader(test_mnist_set)

In [7]:
# Creating train and val data loader
train_loader = torch.utils.data.DataLoader(train_set,batch_size=16,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set,batch_size=16)

In [8]:
from torch import nn

In [9]:
# Model definition
model = nn.Sequential(
    nn.Linear(785,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,16),
    nn.ReLU(),
    nn.Linear(16,2)
)

In [10]:
from torch import optim

In [14]:
# Defining the optimizer
optimizer = optim.Adam(model.parameters(),lr=1e-2)

In [15]:
# Defining the loss function
criteria = nn.MSELoss()

In [16]:
# If GPU is present, the model have to run in the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Train and Val loop
print(f'Device is {device}')
n_epoch = 80
model = model.to(device)
for epoch in range(n_epoch):
  running_loss = 0
  total = 0
  for batch in train_loader:
    x,y = batch
    x = x.cuda()
    optimizer.zero_grad()
    y_hat = model(x.to(device))
    j = criteria(y_hat.double(),y.double().cuda())
    running_loss+=j.item()
    total+=y.size(0)
    model.zero_grad()
    j.backward()
    optimizer.step()
  print(f'Epoch {epoch} Train loss = {running_loss/len(train_loader)}')
  model = model.eval()
  running_loss = 0
  with torch.no_grad():
    for batch in val_loader:
      x,y = batch
      y_hat = model(x.to(device))
      j = criteria(y_hat.double(),y.to(device))
      running_loss+=j.item()
      total+=y.size(0)
  print(f'Epoch {epoch} Validation loss = {running_loss/len(val_loader)}')

Device is cuda


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([16, 2])) that is different to the input size (torch.Size([16, 1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0 Train loss = 0.15493248595080988
Epoch 0 Validation loss = 0.1533671794814644
Epoch 1 Train loss = 0.15431905445510927
Epoch 1 Validation loss = 0.155286526114482
Epoch 2 Train loss = 0.1545496146815351
Epoch 2 Validation loss = 0.15384361760989954
Epoch 3 Train loss = 0.1538175434231588
Epoch 3 Validation loss = 0.15717387299719116
Epoch 4 Train loss = 0.15437669115458988
Epoch 4 Validation loss = 0.15437911006373004
Epoch 5 Train loss = 0.15436437831361088
Epoch 5 Validation loss = 0.15457072071320255
Epoch 6 Train loss = 0.154279105937023
Epoch 6 Validation loss = 0.1540331918396803
Epoch 7 Train loss = 0.1543220037032597
Epoch 7 Validation loss = 0.1549082297192077
Epoch 8 Train loss = 0.154246576931365
Epoch 8 Validation loss = 0.1534672097322463
Epoch 9 Train loss = 0.15392976615535112
Epoch 9 Validation loss = 0.1565150702396145
Epoch 10 Train loss = 0.1539446679102242
Epoch 10 Validation loss = 0.15321383905948802
Epoch 11 Train loss = 0.15302913296176113
Epoch 11 Valid